In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce

In [37]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce

In [38]:
dr = 'data/'

sample = pd.read_csv(dr + 'sample_submission.csv')
train = pd.read_csv(dr + 'train.csv')
test = pd.read_csv(dr + 'test.csv')

In [39]:
X_train = train.copy()
X_test = test.copy()
y_train = X_train.pop('target')

In [40]:
#변수별 unique 개수
[(i,len(train[i].unique())) for i in train.columns]

[('id', 600000),
 ('bin_0', 3),
 ('bin_1', 3),
 ('bin_2', 3),
 ('bin_3', 3),
 ('bin_4', 3),
 ('nom_0', 4),
 ('nom_1', 7),
 ('nom_2', 7),
 ('nom_3', 7),
 ('nom_4', 5),
 ('nom_5', 1221),
 ('nom_6', 1520),
 ('nom_7', 223),
 ('nom_8', 223),
 ('nom_9', 2219),
 ('ord_0', 4),
 ('ord_1', 6),
 ('ord_2', 7),
 ('ord_3', 16),
 ('ord_4', 27),
 ('ord_5', 191),
 ('day', 8),
 ('month', 13),
 ('target', 2)]

In [41]:
cat_all = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 
       'nom_9','ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5','day','month']

binary = ['bin_0','bin_1','bin_2','bin_3', 'bin_4']
low_nom = ['nom_0', 'nom_1','nom_2', 'nom_3', 'nom_4']
high_nom = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
low_ord = ['ord_0', 'ord_1', 'ord_2']
high_ord = ['ord_3', 'ord_4', 'ord_5']
cyclic = ['day','month']

In [42]:
all_cyclic= ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 
       'nom_9','ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']

In [43]:
#submission 제출을 위해 결측값 제거 대신 대체값(최빈값) 사용
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy="most_frequent")

X_train[cat_all] = imp.fit_transform(X_train[cat_all])
X_test[cat_all] = imp.transform(X_test[cat_all])
X_train = X_train.drop(columns=['id'])
X_test = X_test.drop(columns=['id'])

In [44]:
#문자형 -> binary 변환
X_train['bin_3']= np.where(X_train['bin_3']=='F',0,1)
X_train['bin_4']= np.where(X_train['bin_4']=='N',0,1)
X_test['bin_3']= np.where(X_test['bin_3']=='F',0,1)
X_test['bin_4']= np.where(X_test['bin_4']=='N',0,1)

# ● Categorical Feature 유형 고려한 Encoding 기법(가장 높은 성능)

## - LR (Not encoding + Leave One Out Encoding)

In [10]:
# 처음부터 이진 분류로 구성된 binary는 인코딩하지 않고, 나머지는 성능이 가장 좋았던 Leave One Out Encoding 사용

X_train_en, X_test_en = X_train.copy(), X_test.copy()

from category_encoders import LeaveOneOutEncoder 
loe = LeaveOneOutEncoder()

X_train_en[low_nom]=loe.fit_transform(X_train_en[low_nom],y_train)
X_test_en[low_nom]=loe.transform(X_test_en[low_nom])

X_train_en[high_nom]=loe.fit_transform(X_train_en[high_nom],y_train)
X_test_en[high_nom]=loe.transform(X_test_en[high_nom])

X_train_en[low_ord]=loe.fit_transform(X_train_en[low_ord],y_train)
X_test_en[low_ord]=loe.transform(X_test_en[low_ord])

X_train_en[high_ord]=loe.fit_transform(X_train_en[high_ord],y_train)
X_test_en[high_ord]=loe.transform(X_test_en[high_ord])

X_train_en[cyclic]=loe.fit_transform(X_train_en[cyclic],y_train)
X_test_en[cyclic]=loe.transform(X_test_en[cyclic])

# LR_sub(X_train2, X_test2)

# private 성능 0.78339

## - MLP (One Hot Encoding + Leave One Out Encoding)

In [12]:
# low_nom, binary에 one-hot encoding(카디널리티가 적음) 
# high_nom, high_ord, low_ord, cyclic는 성능이 가장 좋았던 Leave One Out Encoding 사용

X_train_en, X_test_en = X_train.copy(), X_test.copy()

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(X_train[low_nom])

ohe_train = pd.DataFrame(ohe.transform(X_train_en[low_nom]))
ohe_test =pd.DataFrame(ohe.transform(X_test_en[low_nom]))

X_train_en = pd.concat([X_train_en,ohe_train],axis=1)
X_test_en = pd.concat([X_test_en,ohe_test],axis=1)
X_train_en.drop(columns=low_nom,inplace=True)
X_test_en.drop(columns=low_nom,inplace=True)

# 
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(X_train[binary])

ohe_train = pd.DataFrame(ohe.transform(X_train_en[binary]))
ohe_test =pd.DataFrame(ohe.transform(X_test_en[binary]))

X_train_en = pd.concat([X_train_en,ohe_train],axis=1)
X_test_en = pd.concat([X_test_en,ohe_test],axis=1)
X_train_en.drop(columns=binary,inplace=True)
X_test_en.drop(columns=binary,inplace=True)

from category_encoders import LeaveOneOutEncoder 
loe = LeaveOneOutEncoder()

X_train_en[high_nom]=loe.fit_transform(X_train_en[high_nom],y_train)
X_test_en[high_nom]=loe.transform(X_test_en[high_nom])


X_train_en[high_ord]=loe.fit_transform(X_train_en[high_ord],y_train)
X_test_en[high_ord]=loe.transform(X_test_en[high_ord])

X_train_en[low_ord]=loe.fit_transform(X_train_en[low_ord],y_train)
X_test_en[low_ord]=loe.transform(X_test_en[low_ord])

X_train_en[cyclic]=loe.fit_transform(X_train_en[cyclic],y_train)
X_test_en[cyclic]=loe.transform(X_test_en[cyclic])

# MLP_sub(X_train_en, X_test_en)

# private 성능 0.78284

## - DT ( K-Fold Target Encoding + Cyclic Data Encoding)

In [45]:
X_train_en, X_test_en = X_train.copy(), X_test.copy()

### KFold TargetEncoding

In [46]:
#high_nom -> Kfold_Te
#high_ord -> Kfold_Te
from sklearn.model_selection import KFold
X_train_imp, X_test_imp = X_train.copy(), X_test.copy()

X_train_imp_te, X_test_imp_te, = X_train_en.copy(), X_test_en.copy()

for c in high_ord+high_nom+low_ord:
    data_tmp = pd.DataFrame({c: X_train_imp[c], 'target': y_train})
    target_mean = data_tmp.groupby(c)['target'].mean()
    X_test_imp_te[c] = X_test_imp[c].map(target_mean)
    tmp = np.repeat(np.nan, X_train_imp.shape[0])

    kf = KFold(n_splits=4, shuffle=True, random_state=0)
    for idx_1, idx_2 in kf.split(X_train_imp):

        target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()

        tmp[idx_2] = X_train_imp[c].iloc[idx_2].map(target_mean)
        
    X_train_imp_te[c] = tmp

In [49]:
#low_nom - > jse
import category_encoders as ce
encoder = ce.JamesSteinEncoder()
X_train_imp_te[low_nom] = encoder.fit_transform(X_train_imp_te[low_nom], y_train)
X_test_imp_te[low_nom] = encoder.transform(X_test_imp_te[low_nom])

In [51]:
#Kfold중 결측된 값을 채우기 위해 전체 평균으로 대체값 처리
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy="mean")

X_train_imp_te[cat_all] = imp.fit_transform(X_train_imp_te[cat_all])
X_test_imp_te[cat_all] = imp.transform(X_test_imp_te[cat_all])

### Cyclic Data encoding

In [53]:
def encode(data, col, max_val):
    data[col]=data[col].astype(int)
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    data.drop(columns=[col],inplace=True)
    return data

In [54]:
encode(pd.DataFrame(X_train_imp_te), 'day', 7)
encode(pd.DataFrame(X_train_imp_te), 'month', 12)
encode(pd.DataFrame(X_test_imp_te), 'day', 7)
encode(pd.DataFrame(X_test_imp_te), 'month', 12)

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,ord_1,ord_2,ord_3,ord_4,ord_5,day,day_sin,day_cos,month_sin,month_cos
0,0.0,0.0,0.0,0.0,1.0,0.195763,0.199762,0.196821,0.181381,0.147237,...,0.159193,0.244003,0.151036,0.222938,0.235641,3,0.433884,-0.900969,-1.000000e+00,-1.836970e-16
1,0.0,0.0,0.0,0.0,1.0,0.183186,0.181912,0.202754,0.209261,0.203884,...,0.159193,0.152205,0.268994,0.187694,0.163537,2,0.974928,-0.222521,-8.660254e-01,-5.000000e-01
2,0.0,0.0,0.0,0.0,1.0,0.195763,0.181912,0.196821,0.209261,0.181100,...,0.189716,0.175404,0.196071,0.187694,0.125748,2,0.974928,-0.222521,1.224647e-16,-1.000000e+00
3,1.0,0.0,0.0,0.0,0.0,0.183186,0.199762,0.196821,0.198004,0.203884,...,0.189716,0.205176,0.265988,0.135848,0.108688,1,0.781831,0.623490,1.224647e-16,-1.000000e+00
4,0.0,0.0,1.0,0.0,1.0,0.183186,0.181912,0.174854,0.181381,0.181100,...,0.172972,0.290762,0.309608,0.175385,0.119081,3,0.433884,-0.900969,1.000000e+00,6.123234e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,0.0,0.0,1.0,1.0,0.0,0.183186,0.213197,0.196821,0.198004,0.181100,...,0.189716,0.175404,0.268994,0.207384,0.182347,2,0.974928,-0.222521,-2.449294e-16,1.000000e+00
399996,0.0,0.0,0.0,0.0,0.0,0.183186,0.199762,0.183463,0.209261,0.181100,...,0.172972,0.138364,0.268994,0.233747,0.177996,1,0.781831,0.623490,5.000000e-01,-8.660254e-01
399997,0.0,0.0,1.0,1.0,0.0,0.183186,0.167577,0.174854,0.171765,0.181100,...,0.232669,0.244003,0.265988,0.201003,0.170383,1,0.781831,0.623490,1.224647e-16,-1.000000e+00
399998,0.0,0.0,0.0,0.0,1.0,0.183186,0.181912,0.183463,0.198004,0.181100,...,0.159193,0.290762,0.182082,0.184068,0.126442,2,0.974928,-0.222521,-5.000000e-01,8.660254e-01


# ● 성능과 상관 없이 시도해본 인코딩

###  LeaveOneOut / Cyclic Encoding

In [59]:
X_train_en, X_test_en = X_train.copy(), X_test.copy()

In [60]:
from category_encoders import LeaveOneOutEncoder 
loe = LeaveOneOutEncoder()

X_train_en[all_cyclic]=loe.fit_transform(X_train_en[all_cyclic],y_train)
X_test_en[all_cyclic]=loe.transform(X_test_en[all_cyclic])

In [61]:
encode(X_train_en,'day', 7)
encode(X_train_en,'month', 12)
encode(X_test_en,'day', 7)
encode(X_test_en,'month', 12)

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day_sin,day_cos,month_sin,month_cos
0,0.194468,0.190482,0.171748,0,1,0.195763,0.205417,0.201104,0.178947,0.140327,...,0.239775,0.159193,0.244003,0.151036,0.222938,0.235641,0.433884,-0.900969,-1.000000e+00,-1.836970e-16
1,0.194468,0.190482,0.171748,0,1,0.183186,0.179694,0.209834,0.219534,0.207375,...,0.145403,0.159193,0.152205,0.268994,0.187694,0.163537,0.974928,-0.222521,-8.660254e-01,-5.000000e-01
2,0.194468,0.190482,0.171748,0,1,0.195763,0.179694,0.201104,0.219534,0.179902,...,0.145403,0.189716,0.175404,0.196071,0.187694,0.125748,0.974928,-0.222521,1.224647e-16,-1.000000e+00
3,0.113365,0.190482,0.171748,0,0,0.183186,0.205417,0.201104,0.202836,0.207375,...,0.145403,0.189716,0.205176,0.265988,0.135848,0.108688,0.781831,0.623490,1.224647e-16,-1.000000e+00
4,0.194468,0.190482,0.228917,0,1,0.183186,0.179694,0.169845,0.178947,0.179902,...,0.145403,0.172972,0.290762,0.309608,0.175385,0.119081,0.433884,-0.900969,1.000000e+00,6.123234e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,0.194468,0.190482,0.228917,1,0,0.183186,0.225464,0.201104,0.202836,0.179902,...,0.186523,0.189716,0.175404,0.268994,0.207384,0.182347,0.974928,-0.222521,-2.449294e-16,1.000000e+00
399996,0.194468,0.190482,0.171748,0,0,0.183186,0.205417,0.181884,0.219534,0.179902,...,0.145403,0.172972,0.138364,0.268994,0.233747,0.177996,0.781831,0.623490,5.000000e-01,-8.660254e-01
399997,0.194468,0.190482,0.228917,1,0,0.183186,0.159894,0.169845,0.165594,0.179902,...,0.145403,0.232669,0.244003,0.265988,0.201003,0.170383,0.781831,0.623490,1.224647e-16,-1.000000e+00
399998,0.194468,0.190482,0.171748,0,1,0.183186,0.179694,0.181884,0.202836,0.179902,...,0.239775,0.159193,0.290762,0.182082,0.184068,0.126442,0.974928,-0.222521,-5.000000e-01,8.660254e-01


### One Hot + Leave One Out Encoding

In [62]:
# low_nom, low_ord, binary에 one-hot encoding(카디널리티가 적음) 
# high_nom, high_ord, cyclic는 성능이 가장 좋았던 Leave One Out Encoding 사용

X_train_en, X_test_en = X_train.copy(), X_test.copy()

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(X_train[low_nom])

ohe_train = pd.DataFrame(ohe.transform(X_train_en[low_nom]))
ohe_test =pd.DataFrame(ohe.transform(X_test_en[low_nom]))

X_train_en = pd.concat([X_train_en,ohe_train],axis=1)
X_test_en = pd.concat([X_test_en,ohe_test],axis=1)
X_train_en.drop(columns=low_nom,inplace=True)
X_test_en.drop(columns=low_nom,inplace=True)

# 
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(X_train[binary])

ohe_train = pd.DataFrame(ohe.transform(X_train_en[binary]))
ohe_test =pd.DataFrame(ohe.transform(X_test_en[binary]))

X_train_en = pd.concat([X_train_en,ohe_train],axis=1)
X_test_en = pd.concat([X_test_en,ohe_test],axis=1)
X_train_en.drop(columns=binary,inplace=True)
X_test_en.drop(columns=binary,inplace=True)

# 
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(X_train[low_ord])

ohe_train = pd.DataFrame(ohe.transform(X_train_en[low_ord]))
ohe_test =pd.DataFrame(ohe.transform(X_test_en[low_ord]))

X_train_en = pd.concat([X_train_en,ohe_train],axis=1)
X_test_en = pd.concat([X_test_en,ohe_test],axis=1)
X_train_en.drop(columns=low_ord,inplace=True)
X_test_en.drop(columns=low_ord,inplace=True)

from category_encoders import LeaveOneOutEncoder 
loe = LeaveOneOutEncoder()

X_train_en[high_nom]=loe.fit_transform(X_train_en[high_nom],y_train)
X_test_en[high_nom]=loe.transform(X_test_en[high_nom])


X_train_en[high_ord]=loe.fit_transform(X_train_en[high_ord],y_train)
X_test_en[high_ord]=loe.transform(X_test_en[high_ord])


X_train_en[cyclic]=loe.fit_transform(X_train_en[cyclic],y_train)
X_test_en[cyclic]=loe.transform(X_test_en[cyclic])

# MLP_sub(X_train_en, X_test_en)

# private 성능 0.78179

## Submission

In [12]:
def LR_sub(X_train,X_test):
    LR = LogisticRegression()
    LR.fit(X_train, y_train)
    predict = LR.predict_proba(X_test)[:,1]
    submission = pd.DataFrame(predict).reset_index()
    submission['index'] = submission['index']+600000
    submission.columns = ['id','target']
    submission.set_index('id')
    
    t = pd.Timestamp.now()
    fname = f"submission_LR{t.month:02}{t.day:02}_{t.hour:02}{t.minute:02}.csv"
    submission.to_csv(fname,index=False)

In [56]:
def DT_sub(X_train,X_test):
    DT = DecisionTreeClassifier(max_depth = 12, random_state=0)
    DT.fit(X_train, y_train)
    predict = DT.predict_proba(X_test)[:,1]
    submission = pd.DataFrame(predict).reset_index()
    submission['index'] = submission['index']+600000
    submission.columns = ['id','target']
    submission.set_index('id')
    
    t = pd.Timestamp.now()
    fname = f"submission_DT{t.month:02}{t.day:02}_{t.hour:02}{t.minute:02}.csv"
    submission.to_csv(fname,index=False)
    return predict

In [14]:
def MLP_sub(X_train,X_test):
    MLP = MLPClassifier(hidden_layer_sizes=(50,))
    MLP.fit(X_train, y_train)
    predict = MLP.predict_proba(X_test)[:,1]
    submission = pd.DataFrame(predict).reset_index()
    submission['index'] = submission['index']+600000
    submission.columns = ['id','target']
    submission.set_index('id')
    
    t = pd.Timestamp.now()
    fname = f"submission_MLP{t.month:02}{t.day:02}_{t.hour:02}{t.minute:02}.csv"
    submission.to_csv(fname,index=False)
    return predict